##

# Determine Bias of Vine Reviews
Using your knowledge of PySpark, Pandas, or SQL, you’ll determine if there is any bias towards reviews that were written as part of the Vine program. For this analysis, you'll determine if having a paid Vine review makes a difference in the percentage of 5-star reviews.

## Import Data and Dependencies

In [53]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [54]:
# read in vine table csv which was exported from PgAdmin
import matplotlib.pyplot as plt
vine_table_df = pd.read_csv('resources/vine_table.csv')
vine_table_df.head(5)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,RTIS3L2M1F5SM,5,0,0,N,Y
1,R1ZV7R40OLHKD,5,0,0,N,Y
2,R3BH071QLH8QMC,1,0,1,N,Y
3,R127K9NTSXA2YH,3,0,0,N,Y
4,R32ZWUXDJPW27Q,4,0,0,N,Y


## Preview Analysis - Examine Data Types & Basic Stats

In [55]:
# Look for Patterns
vine_table_df.describe()


,star_rating,helpful_votes,total_votes
count,1.785997e+06,1.785997e+06,1.785997e+06
mean,4.059893e+00,2.259349e+00,3.757369e+00
std,1.355812e+00,1.917129e+01,2.245439e+01
min,1.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,0.000000e+00,0.000000e+00
50%,5.000000e+00,0.000000e+00,1.000000e+00
75%,5.000000e+00,1.000000e+00,3.000000e+00
max,5.000000e+00,1.049800e+04,1.078000e+04


In [56]:
# Look at data types
vine_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785997 entries, 0 to 1785996
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   review_id          object
 1   star_rating        int64 
 2   helpful_votes      int64 
 3   total_votes        int64 
 4   vine               object
 5   verified_purchase  object
dtypes: int64(3), object(3)
memory usage: 81.8+ MB


In [57]:
# Count Rows
vine_table_df["review_id"].count()

1785997

## 1. Filter By Total Votes
Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.


In [58]:
total_votes_df = vine_table_df[vine_table_df['total_votes'] >= 20]
total_votes_df.head(n=5)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
289,R3EZ0EPYLDA34S,1,14,31,N,Y
483,R2FJ94555FZH32,2,55,60,N,N


In [59]:
total_votes_df["review_id"].count()

65379

## 2. Filter By Most Helpful Votes
Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

- If you use the SQL option below, you’ll need to cast your columns as floats using WHERE CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5.

In [60]:
helpful_votes_df = total_votes_df.loc[total_votes_df.helpful_votes / total_votes_df.total_votes >= 0.5]
helpful_votes_df.head(n=5)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
483,R2FJ94555FZH32,2,55,60,N,N
537,R1U3AR67RE273L,1,51,65,N,Y


In [61]:
helpful_votes_df["review_id"].count()

40565

## 3. Filter By Paid Reviews
Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [62]:
paid_reviews_df = helpful_votes_df[helpful_votes_df['vine'] == 'Y']
paid_reviews_df.head(n=5)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
32611,R3KKUSGFZWSUIY,5,56,63,Y,N
33112,R10FO5UKKVZBK2,3,23,23,Y,N
69680,RM4KSGEOR7MU1,5,19,24,Y,N
155361,RG7VRMYLEXD23,4,22,26,Y,N
239327,R11O4YSCPSNL6L,3,20,26,Y,N


In [63]:
paid_count = paid_reviews_df["review_id"].count()
paid_count

94

## 4. Filter By Unpaid Reviews
Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [64]:
unpaid_reviews_df = helpful_votes_df[helpful_votes_df['vine'] == 'N']
unpaid_reviews_df.head(n=5)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
483,R2FJ94555FZH32,2,55,60,N,N
537,R1U3AR67RE273L,1,51,65,N,Y


In [65]:
unpaid_count = unpaid_reviews_df["review_id"].count()
unpaid_count

40471

## 5. 5-Star Review Analysis
Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

### Total Number of Paid vs Unpaid Reviews

In [66]:
vine_count = paid_count + unpaid_count
print(f"Total Number of Reviews: {vine_count}")
print(f"Number of Paid Views: {paid_count}")
print(f"Number of Unpaid Views: {unpaid_count}")
print(f"Out of {vine_count} reviews, {paid_count} were paid.")


Total Number of Reviews: 40565
Number of Paid Views: 94
Number of Unpaid Views: 40471
Out of 40565 reviews, 94 were paid.


### Total Number of 5-Star Paid & Unpaid Reviews

In [67]:
five_star_paid = len(paid_reviews_df[paid_reviews_df['star_rating']==5])

five_star_unpaid = len(unpaid_reviews_df[unpaid_reviews_df['star_rating']==5])

In [68]:
print(f"Number of 5-Star Paid Views: {five_star_paid}")
print(f"Number of Paid Views: {paid_count}")
print(f"Number of 5-Star UnPaid Views: {five_star_unpaid}")
print(f"Number of Paid Views: {unpaid_count}")

Number of 5-Star Paid Views: 48
Number of Paid Views: 94
Number of 5-Star UnPaid Views: 15663
Number of Paid Views: 40471


### Percentage Of 5-Star Reviews

In [69]:
print(f"Percentage of 5-Star Paid Reviews: {round(five_star_paid / paid_count, 2)}")
print(f"Percentage of 5-Star Unpaid Reviews: {round(five_star_unpaid / unpaid_count, 2)}")


Percentage of 5-Star Paid Reviews: 0.51
Percentage of 5-Star Unpaid Reviews: 0.39


### Average 5-Star Rating

In [70]:
paid_avg = round(paid_reviews_df['star_rating'].mean(),2)

unpaid_avg = round(unpaid_reviews_df['star_rating'].mean(),2)

In [71]:
print(f"Average 5-Star Paid Review: {paid_avg}")
print(f"Average 5-Star Paid Review: {unpaid_avg}")


Average 5-Star Paid Review: 4.2
Average 5-Star Paid Review: 3.35
